In [1]:
import pandas as pd
import nltk
import re
import string
import requests
import xlrd
import xlwt
# import config
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sb
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
sb.set(style='white')

# Subjectivity Detection

In [113]:
separated_df = pd.read_csv('./reddit_post_with_separated_comments_with_datetime.csv')

In [114]:
separated_df

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,1. Don't normalize what other people do or wha...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:44:26,https://www.reddit.com/r/mentalhealth/comments...
1,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Never compare yourself to anybody else. Your l...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:28:51,https://www.reddit.com/r/mentalhealth/comments...
2,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I can relate but I do like to remind myself th...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:46:40,https://www.reddit.com/r/mentalhealth/comments...
3,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I'm almost certain most people lie about the a...,mentalhealth,positive,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:02:33,https://www.reddit.com/r/mentalhealth/comments...
4,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Find yourself a nice woman and try to have a r...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:05:27,https://www.reddit.com/r/mentalhealth/comments...
...,...,...,...,...,...,...,...,...,...
16940,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",For almost a year I thought my partner was in ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 02:51:35,https://www.reddit.com/r/schizophrenia/comment...
16941,Thought Projection,"Hello, as a Schizophrenic one of my only illus...","One thing that could be helpful, is if people ...",schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-17 02:47:02,https://www.reddit.com/r/schizophrenia/comment...
16942,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",[removed],schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 00:34:53,https://www.reddit.com/r/schizophrenia/comment...
16943,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",I have that problem too- it’s just what it is ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 07:59:11,https://www.reddit.com/r/schizophrenia/comment...


In [53]:
def remove_punctuations(review: str) -> str:
    words: list[str] = review.split()
    table: dict[int, int] = str.maketrans('', '', string.punctuation)
    stripped: list[str] = [w.translate(table) for w in words]
    return " ".join(stripped)

def normalize(review: str) -> str:
    return review.lower()

def remove_stopwords(review: str, stopwords: set[str]) -> str:
    words: list[str] = review.split()
    words = [w for w in words if not w in stopwords]
    return " ".join(words)


def stemming(review: str) -> str:
    tokens: list[str] = word_tokenize(review)
    porter: PorterStemmer = PorterStemmer()
    stemmed: list[str] = [porter.stem(word) for word in tokens]
    return " ".join(stemmed)

def lemmatize(review: str) -> str:
    lemmatizer = WordNetLemmatizer()
    tokens: list[str] = word_tokenize(review)
    lemmatized: list[str] = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

def depure_data(data):
    
    #remove url with a RegEx
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    #remove email address
    data = re.sub('\S*@\S*\s?', '', data)

    #remove new line characters
    data = re.sub('\s+', ' ', data)

    #remove single quotes
    data = re.sub("\'", "", data)
        
    return data
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [50]:
separated_df['post_comment'].fillna('', inplace=True)

# Convert all values in the 'text' column to strings
separated_df['post_comment'] = separated_df['post_comment'].astype(str)

In [54]:
separated_df['post_comment_processed'] = separated_df['post_comment'] \
    .apply(remove_punctuations) \
    .apply(normalize) \
    .apply(lambda x: remove_stopwords(x, set(stopwords.words('english')))) \
    .apply(lemmatize) \
    .apply(remove_emojis) \
    .apply(depure_data)


In [56]:
separated_df['post_comment_processed'] = separated_df['post_comment_processed'].str.replace(' ’ ','’')

In [58]:
separated_df.to_excel('reddit_post_with_separated_comments_with_datetime.xlsx')

In [67]:
LANG = 'en'
APIURL = 'https://sentic.net/api/' + LANG + '/' + config.api_key + '.py?text='
FILENAME = 'reddit_post_with_separated_comments_with_datetime'

wb = xlrd.open_workbook(FILENAME + '.xls')
sheet = wb.sheet_by_index(0)
new_wb = xlwt.Workbook(style_compression=2)
new_sheet = new_wb.add_sheet('labeled')

count = 0
for row in range(0, 16946):
    text = str(sheet.cell_value(row, 0))

    # Replace certain characters
    for c in [';', '&', '#', '{', '}']:
        text = text.replace(c, ':')

    label = str(requests.get(APIURL + text).content)[2:-3]
    new_sheet.write(count, 0, text)
    
    # Write labels with formatting
    if label == 'NEGATIVE': 
        new_sheet.write(count, 1, label, xlwt.easyxf('font: color red; align: horiz center'))
        print(text + ': \033[91m' + label + '\033[0;0m')
    elif label == 'POSITIVE':
        new_sheet.write(count, 1, label, xlwt.easyxf('font: color green; align: horiz center'))
        print(text + ': \033[92m' + label + '\033[0;0m')
    else:
        new_sheet.write(count, 1, label, xlwt.easyxf('align: horiz center'))
        print(text + ': ' + label)
    count += 1
    
new_wb.save(FILENAME + '_labeled.xls')


1 dont normalize people whats around u everyone fucked 2 wan na get laid quit worrying shoot shot every girl u find attractive one u least u enough go u take u fake fake 60 year real honest amazing confident assume youre dude woman different u want shit keep hammering away snag one lol u got yo: SUBJECTIVE
never compare anybody else life life story story live way want: SUBJECTIVE
relate like remind least i’m risking std’s pregnancy nude leaked toxic people sex later try haunt plus don’t care hook ups casual sex don’t like idea trying sleep much people possible rather sleep someone love cherishes sleep someone want physically get pleasure think finding someone actually genuine relationship better casual hook ups genuine relationship you’re viewed person hookup you’re viewed someone soon later redundant: SUBJECTIVE
im almost certain people lie amount time sex seems cool pipe laying manly man idk equivalent woman im guy im talking experience dont focus much others say dont beat youre 20 y